### Цель исследования:
Проанализировать базу данных с информацией о книгах, авторах книг, издательствах и пользовательских обзорах книг.
Это поможет нам в перспективе сфорировать ценностное предложение нового сервиса для любитей читать.  

###### План работы:
- 1) Импортировать необходимые библиотеки. Создать коннекцию к базе.
- 2) Знакомство с данными. Исследовать таблицы (вывести первые строки).
- 3) Посчитайте количество книг, выпущенных после 1 января 2000 года.
- 4) Посчитать количество пользовательских обзоров и среднюю оценку для каждой книги.
- 5) Определить издательство, которое издало наибольшее число книг толще 50 страниц.
- 6) Определить автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более пользовательскими оценками.
- 7) Посчитайть среднее количество текстовых обзоров пользователей, которые поставили более чем по 50 оценок.

###### Данных:
Таблица **books**:

Содержит данные о книгах:
* book_id — идентификатор книги;
* author_id — идентификатор автора;
* title — название книги;
* num_pages — количество страниц;
* publication_date — дата публикации книги;
* publisher_id — идентификатор издателя.

Таблица **authors**:

Содержит данные об авторах:
* author_id — идентификатор автора;
* author — имя автора.

Таблица **publishers**:

Содержит данные об издательствах:
* publisher_id — идентификатор издательства;
* publisher — название издательства;

Таблица **ratings**:

Содержит данные о пользовательских оценках книг:
* rating_id — идентификатор оценки;
* book_id — идентификатор книги;
* username — имя пользователя, оставившего оценку;
* rating — оценка книги.

Таблица **reviews**:

Содержит данные о пользовательских обзорах:
* view_id — идентификатор обзора;
* book_id — идентификатор книги;
* username — имя автора обзора;
* text — текст обзора.

###### 1) Импортировать необходимые библиотеки. Создать коннекцию к базе.

In [1]:
import warnings 
import pandas as pd
from sqlalchemy import create_engine
warnings.filterwarnings("ignore")
pd.options.display.max_rows = 999
pd.set_option('display.max_colwidth', -1)

In [2]:
db_config = {'user': 'praktikum_student', 
             'pwd': 'Sdf4$2;d-d30pp',
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, 
             'db': 'data-analyst-final-project-db'} 
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])
query_books = """
SELECT *
FROM books;  
"""
engine = create_engine(connection_string, connect_args={'sslmode':'require'})
books = pd.io.sql.read_sql(query_books, con = engine)

query_authors = """
SELECT *
FROM authors;  
"""
engine = create_engine(connection_string, connect_args={'sslmode':'require'})
authors = pd.io.sql.read_sql(query_authors, con = engine)

query_publishers = """
SELECT *
FROM publishers;  
"""
engine = create_engine(connection_string, connect_args={'sslmode':'require'})
publishers = pd.io.sql.read_sql(query_publishers, con = engine)

query_ratings = """
SELECT *
FROM ratings;  
"""
engine = create_engine(connection_string, connect_args={'sslmode':'require'})
ratings = pd.io.sql.read_sql(query_ratings, con = engine)

query_reviews = """
SELECT *
FROM reviews;  
"""
engine = create_engine(connection_string, connect_args={'sslmode':'require'})
reviews = pd.io.sql.read_sql(query_reviews, con = engine)

###### 2) Знакомство с данными. Исследовать таблицы (вывести первые строки).

In [3]:
books.head(5)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope #1),322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before Columbus,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
authors.head(5)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:
publishers.head(5)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [6]:
ratings.head(5)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [7]:
reviews.head(5)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. Over provide race technology continue these.
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Among admit investment argue security.
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but person sport treatment industry. Kitchen decision deep the. Social party body the.
3,4,3,johnsonamanda,Finally month interesting blue could nature cultural bit. Prepare beat finish grow that smile teach. Dream me play near.
4,5,3,scotttamara,Nation purpose heavy give wait song will. List dinner another whole positive radio fast. Music staff many green.


###### 3) Посчитать количество книг, выпущенных после 1 января 2000 года.

In [8]:
query_books_count_books_20000101 = """
SELECT COUNT(book_id) as count_books
FROM books
WHERE publication_date  > '2000-01-01';  
"""
engine = create_engine(connection_string, connect_args={'sslmode':'require'})
books_count = pd.io.sql.read_sql(query_books_count_books_20000101, con = engine)

In [9]:
books_count

,count_books
0,819


###### Вывод:
Количество книг, выпущенных после 1 января 2000 года - 819 шт.

###### 4) Посчитать количество пользовательских обзоров и среднюю оценку для каждой книги.

In [10]:
counts_rewiews_and_avg_ratings = """
SELECT books.book_id as id,
       books.title as name,
       COUNT(DISTINCT reviews.review_id) as reviews_number,
       AVG(ratings.rating) as avg_ratings 
FROM books
INNER JOIN reviews ON reviews.book_id = books.book_id
INNER JOIN ratings ON ratings.book_id = books.book_id
GROUP BY
books.book_id,
books.title
ORDER BY
COUNT(reviews.review_id) DESC;  
"""
engine = create_engine(connection_string, connect_args={'sslmode':'require'})
rewiews_avg_ratings_count = pd.io.sql.read_sql(counts_rewiews_and_avg_ratings, con = engine)

In [11]:
rewiews_avg_ratings_count.head()

,id,name,reviews_number,avg_ratings
0,948,Twilight (Twilight #1),7,3.662500
1,750,The Hobbit or There and Back Again,6,4.125000
2,673,The Catcher in the Rye,6,3.825581
3,302,Harry Potter and the Prisoner of Azkaban (Harry Potter #3),6,4.414634
4,299,Harry Potter and the Chamber of Secrets (Harry Potter #2),6,4.287500


In [12]:
reviews['review_id'].count()

2793

In [13]:
rewiews_avg_ratings_count['reviews_number'].sum()

2793

###### Вывод:
- Средние оценки для книг с самым большим количеством обзоров (ТОП - 10) - варьируются от 3.5 до 4.5
- Наибольшее количество обзоров получилитакие книги как Twilight, The Hobbit or There and Back Again, The Catcher in the Rye с количеством обзоров 7, 6, 6 соответственно.

###### 5) Определить издательство, которое издало наибольшее число книг толще 50 страниц.

In [14]:
request_publisher = """
SELECT books.publisher_id as id,
       publishers.publisher as name_publisher,
       COUNT(books.book_id) as cnt_books
FROM books
INNER JOIN publishers ON publishers.publisher_id  = books.publisher_id 
WHERE
books.num_pages >= 50
GROUP BY
books.publisher_id,
publishers.publisher
ORDER BY
COUNT(books.book_id) DESC
LIMIT
5;
"""
engine = create_engine(connection_string, connect_args={'sslmode':'require'})
publisher = pd.io.sql.read_sql(request_publisher, con = engine)

In [15]:
publisher

,id,name_publisher,cnt_books
0,212,Penguin Books,42
1,309,Vintage,31
2,116,Grand Central Publishing,25
3,217,Penguin Classics,24
4,35,Bantam,19


###### Вывод:
Издательство, которое издало наибольшее число книг толще 50 страниц - Penguin Books

###### 6) Определить автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более пользовательскими оценками.

In [16]:
request_author_ratings = """
SELECT books.author_id as id,
       authors.author as name_author,
       AVG(ratings.rating) as avg_ratings 
FROM books
INNER JOIN authors ON authors.author_id = books.author_id
INNER JOIN ratings ON ratings.book_id = books.book_id 
WHERE ratings.book_id IN
      (SELECT ratings.book_id 
      FROM ratings
      GROUP BY
      ratings.book_id
      HAVING 
      COUNT(ratings.rating_id) >= 50)
GROUP BY
books.author_id,
authors.author
ORDER BY
AVG(ratings.rating) DESC
LIMIT
5;
"""
engine = create_engine(connection_string, connect_args={'sslmode':'require'})
author_ratings_avg_ratings_book = pd.io.sql.read_sql(request_author_ratings, con = engine)

In [17]:
author_ratings_avg_ratings_book

,id,name_author,avg_ratings
0,236,J.K. Rowling/Mary GrandPré,4.287097
1,402,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,240,J.R.R. Tolkien,4.246914
3,376,Louisa May Alcott,4.192308
4,498,Rick Riordan,4.080645


###### Вывод:
Автор с самой высокой средней оценкой книг (4.3) - J.K. Rowling/Mary GrandPré. Примечание: учитывались только книги с 50 и более пользовательскими оценками.

###### 7) Посчитать среднее количество текстовых обзоров пользователей, которые поставили более чем по 50 оценок.

In [18]:
request_avg_reviews = """
SELECT COUNT(reviews.review_id) / COUNT(DISTINCT reviews.username) as reviews  
FROM ratings
LEFT JOIN reviews ON reviews.review_id = ratings.rating_id 
WHERE reviews.username IN 
      (SELECT ratings.username
      FROM ratings
      GROUP BY
      ratings.username
      HAVING 
      COUNT(ratings.rating_id) > 50);
"""
engine = create_engine(connection_string, connect_args={'sslmode':'require'})
avg_reviews = pd.io.sql.read_sql(request_avg_reviews, con = engine)

In [19]:
avg_reviews

,reviews
0,24


###### Вывод:
Среднее количество текстовых обзоров пользователей, которые поставили более чем по 50 оценок - 24 обзора.